# **Introduction to Task Guardrails in CrewAI**

This notebook will demonstrate using CrewAI Task Guardrails — a powerful way to add custom validation logic to your agent workflows.

Just like bumpers in bowling, guardrails help your AI agents stay on track by validating the output of a task and deciding whether it needs to retry.

We’ll show you how to:
*   Write a custom guardrail function
*   Attach it to a task
*   Automatically retry until the output meets your criteria

Let’s get started! 🚀

## Setup: Imports and API Key
Before we define our agent and guardrail logic, we import the required modules from CrewAI and initialize the LLM.

We'll also use a search tool (**SerperDevTool**) and the **TaskOutput** object to handle validation logic.

In [1]:
%uv pip install -U -q crewai crewai-tools

Note: you may need to restart the kernel to use updated packages.


In [1]:
from crewai import Agent, Task, LLM, Crew
from typing import Tuple, Union, Dict, Any
from crewai import TaskOutput
from datetime import date
from crewai_tools import SerperDevTool

/Users/anup/Desktop/Anup/crew_ai_project/.venv_manual/lib/python3.12/site-packages/pydantic/_internal/_config.py:323: PydanticDeprecatedSince20: Support for class-based `config` is deprecated, use ConfigDict instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.11/migration/
  warnings.warn(DEPRECATION_MESSAGE, DeprecationWarning)


In [2]:
# Filter out deprecation warnings
import warnings
warnings.filterwarnings("ignore")
warnings.filterwarnings("ignore", category=SyntaxWarning, module="pysbd")

In [4]:
from dotenv import load_dotenv

load_dotenv()

True

In [5]:
import os

SERPER_API_KEY = os.getenv('SERPER_API_KEY')
OPENAI_API_KEY = os.getenv('OPENAI_API_KEY')

if SERPER_API_KEY and OPENAI_API_KEY:
    os.environ['SERPER_API_KEY'] = SERPER_API_KEY
    os.environ['OPENAI_API_KEY'] = OPENAI_API_KEY
    print('✅ API keys set successfully!')
else:
    raise ValueError('Please enter both SERPER and OPENAI API keys')

✅ API keys set successfully!


## Custom Guardrail Function
Here’s where the magic happens!

Custom Validation Function (validate_support_response):

Checks word count (50-300 words range)
Validates required elements: acknowledgment, solution offered, next steps
Screens for prohibited unprofessional phrases
Ensures professional closing
More comprehensive validation logic

In [10]:
# custom function
def validate_support_response(result: TaskOutput) -> Tuple[bool, Any]:
    """Validate customer support response meets company standards."""
    try:
        response_text = result.raw.strip().lower()
        original_text = result.raw.strip()
        
        # Check response length (should be between 50-300 words)
        try:
            word_count = len(original_text.split())
            print(f"Word count: {word_count}")
            if word_count < 50:
                return (False, "Support response too short (minimum 50 words)")
            if word_count > 300:
                return (False, "Support response too long (maximum 300 words)")
        except Exception as wc_error:
            print(f"Error during word count check: {wc_error}")
            return (False, f"Error during word count check: {wc_error}")

        # Check for required elements
        required_elements = {
            "acknowledgment": ["thank", "appreciate", "understand", "sorry"],
            "solution_offered": ["solution", "resolve", "fix", "help", "assist"],
            "next_steps": ["next", "follow", "contact", "reach out", "will"]
        }
        
        missing_elements = []
        for element, keywords in required_elements.items():
            if not any(keyword in response_text for keyword in keywords):
                missing_elements.append(element)
        
        if missing_elements:
            return (False, f"Missing required elements: {', '.join(missing_elements)}")

        # Check for prohibited phrases/tone
        prohibited_phrases = [
            "not my problem", "can't help", "impossible", "never", 
            "you should have", "that's wrong", "you're mistaken"
        ]
        
        for phrase in prohibited_phrases:
            if phrase in response_text:
                return (False, f"Contains prohibited phrase: '{phrase}'")

        # Check for professional closing
        professional_closings = [
            "sincerely", "best regards", "thank you", "regards", 
            "happy to help", "let me know", "feel free"
        ]
        
        has_professional_closing = any(closing in response_text for closing in professional_closings)
        if not has_professional_closing:
            return (False, "Missing professional closing")

        print("✅ All validation checks passed!")
        return (True, original_text)
        
    except Exception as e:
        print(f"Unexpected error during validation: {e}")
        return (False, f"Validation error: {e}")

## Define your Crew (Agent + Task + Tools)
Agent Role Changed:

From "Blog Writer" to "Customer Support Specialist"
Specialized backstory with customer service experience
Tools include web scraping for policy lookups


Task Structure:

Analyzes customer inquiries with context (customer name, account type, issue category)
Generates professional support responses
Different output requirements focused on customer service standards


Practical Features:

Handles real customer scenarios (billing issues, account problems)
Maintains empathetic tone validation
Includes follow-up instructions
Professional communication standards



This use case is more business-applicable and demonstrates how CrewAI can be used for customer service automation with quality guardrails to ensure professional, helpful responses that meet company standards.
You could easily adapt this pattern for other use cases like:

Legal document review (validate citations, language, completeness)
Marketing copy (validate brand voice, call-to-actions, compliance)
Technical documentation (validate accuracy, completeness, formatting)

In [12]:
# CrewAI code
from crewai import Agent, Task, Crew, LLM
from crewai_tools import SerperDevTool, ScrapeWebsiteTool

# LLM to be used by the agent(s)
llm = LLM(model="gpt-4o-mini", api_key=OPENAI_API_KEY)

# Customer Support Agent
support_agent = Agent(
    role="Customer Support Specialist",
    goal="Provide helpful, professional, and empathetic customer support responses",
    backstory="""You are an experienced customer support specialist with 5+ years 
    of experience helping customers resolve issues. You're known for your patience, 
    clear communication, and ability to de-escalate situations while finding solutions.""",
    tools=[SerperDevTool(), ScrapeWebsiteTool()],
    llm=llm,
    verbose=True
)

# Customer Support Task
support_task = Task(
    description="""
    Analyze the customer inquiry: "{customer_message}"
    
    Customer details:
    - Name: {customer_name}
    - Account type: {account_type}
    - Issue category: {issue_category}
    
    Provide a professional customer support response that:
    1. Acknowledges the customer's concern
    2. Offers a clear solution or next steps
    3. Maintains a helpful and empathetic tone
    4. Includes appropriate follow-up instructions
    """,
    expected_output="""A professional customer support response that:
    - Is between 50-300 words
    - Acknowledges the customer's issue
    - Provides a clear solution or action plan
    - Uses empathetic and professional language
    - Includes next steps or follow-up information
    - Ends with a professional closing
    """,
    agent=support_agent,
    guardrail=validate_support_response,  # Add the validation guardrail
    max_retries=3  # Maximum number of retries if validation fails
)

# Create the crew
support_crew = Crew(
    agents=[support_agent],
    tasks=[support_task],
    verbose=True
)

## Launch the Crew
Finally, we bundle the agent and task into a **Crew**, pass in some inputs (*prompt* and *year*), and run it using **.kickoff()**.

In [13]:
inputs = {
        "customer_message": "I've been charged twice for my subscription this month and I can't access my premium features. This is very frustrating!",
        "customer_name": "Sarah Johnson", 
        "account_type": "Premium",
        "issue_category": "Billing"
    }
    
# Execute the crew
result = support_crew.kickoff(inputs=inputs)
print("\n" + "="*50)
print("FINAL CUSTOMER SUPPORT RESPONSE:")
print("="*50)
print(result)

╭──────────────────────────────────────────── Crew Execution Started ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Started                                                                                         │
│  Name: crew                                                                                                     │
│  ID: e697c7fb-c9db-40c7-b3ef-db43ca0f9ee1                                                                       │
│  Tool Args:                                                                                                     │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭─────────────────────────────────────────────── 🤖 Agent Started ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Customer Support Specialist                                                                             │
│                                                                                                                 │
│  Task:                                                                                                          │
│      Analyze the customer inquiry: "I've been charged twice for my subscription this month and I can't access   │
│  my premium features. This is very frustrating!"                                                                │
│                                                                                                                 │
│      Customer details:                                                                                          │
│      - Name: Sarah Johnson                                                                                      │
│      - Account type: Premium                                                                                    │
│      - Issue category: Billing                                                                                  │
│                                                                                                                 │
│      Provide a professional customer support response that:                                                     │
│      1. Acknowledges the customer's concern                                                                     │
│      2. Offers a clear solution or next steps                                                                   │
│      3. Maintains a helpful and empathetic tone                                                                 │
│      4. Includes appropriate follow-up instructions                                                             │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

Output()

╭──────────────────────────────────────────── 🔧 Agent Tool Execution ────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Customer Support Specialist                                                                             │
│                                                                                                                 │
│  Thought: I need to gather more information about the billing issue Sarah Johnson is facing. Her inquiry        │
│  suggests that there may be technical details or updates on this situation that I should review.                │
│                                                                                                                 │
│  Using Tool: Search the internet with Serper                                                                    │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭────────────────────────────────────────────────── Tool Input ───────────────────────────────────────────────────╮
│                                                                                                                 │
│  "{\"search_query\": \"issue double charge subscription premium account access\"}"                              │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

...

Word count: 193
✅ All validation checks passed!


╭────────────────────────────────────────────── 🛡️ Guardrail Success ──────────────────────────────────────────────╮
│                                                                                                                 │
│  Guardrail Passed                                                                                               │
│  Name: Validation Successful                                                                                    │
│  Status: ✅ Validated                                                                                           │
│  Attempts: 1                                                                                                    │
│  Tool Args:                                                                                                     │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭──────────────────────────────────────────────── Task Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Task Completed                                                                                                 │
│  Name: 46edf7ed-589f-4d79-a19a-808b8dc8b789                                                                     │
│  Agent: Customer Support Specialist                                                                             │
│  Tool Args:                                                                                                     │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭──────────────────────────────────────────────── Crew Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Completed                                                                                       │
│  Name: crew                                                                                                     │
│  ID: e697c7fb-c9db-40c7-b3ef-db43ca0f9ee1                                                                       │
│  Tool Args:                                                                                                     │
│  Final Output: Dear Sarah,                                                                                      │
│                                                                                                                 │
│  Thank you for reaching out, and I sincerely apologize for the frustration you've experienced with being        │
│  charged twice for your subscription and unable to access your premium features. I understand how important it  │
│  is to have these issues resolved quickly.                                                                      │
│                                                                                                                 │
│  To start, I recommend checking if you have multiple active subscriptions under the same account, as this       │
│  could potentially lead to being charged twice. You can do this by reviewing your transaction history in your   │
│  account settings.                                                                                              │
│                                                                                                                 │
│  If you do indeed have duplicate subscriptions, please cancel any extras to stop the additional charges. If     │
│  everything appears correct, I suggest contacting our billing support team directly. They will have the         │
│  ability to look into your account more closely and assist with any necessary refunds for the duplicate         │
│  charge.                                                                                                        │
│                                                                                                                 │
│  You can reach them at [insert billing support contact details] or through your account page under the support  │
│  section. If you continue to have trouble accessing your premium features after this is resolved, please let    │
│  us know, and we'll be here to assist you further.                                                              │
│                                                                                                                 │
│  Thank you for your patience as we work to resolve this matter.                                                 │
│                                                                                                                 │
│  Best regards,                                                                                                  │
│  [Your Name]                                                                                                    │
│  Customer Support Specialist                                                                                    │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯


FINAL CUSTOMER SUPPORT RESPONSE:
Dear Sarah,

Thank you for reaching out, and I sincerely apologize for the frustration you've experienced with being charged twice for your subscription and unable to access your premium features. I understand how important it is to have these issues resolved quickly.

To start, I recommend checking if you have multiple active subscriptions under the same account, as this could potentially lead to being charged twice. You can do this by reviewing your transaction history in your account settings.

If you do indeed have duplicate subscriptions, please cancel any extras to stop the additional charges. If everything appears correct, I suggest contacting our billing support team directly. They will have the ability to look into your account more closely and assist with any necessary refunds for the duplicate charge.

You can reach them at [insert billing support contact details] or through your account page under the support section. If you continue to hav

## Displaying the Result
Once the Crew finishes executing, we display the final blog post using Markdown formatting:

In [16]:
from IPython.display import display, Markdown
display(Markdown(results.raw))

# CrewAI in 2025
## Transforming Automation with AI

**CrewAI** emerges as a leading platform, streamlining workflows across industries. With its powerful AI agents, users can deploy automated workflows effortlessly. In 2025, expect improved performance and flexibility, reshaping how businesses operate.

## Conclusion

And that’s it! You’ve now seen how to:

*   Write and register a task guardrail in CrewAI
*   Validate custom output logic
*   Automatically rerun tasks that don’t meet your standards

This makes your agents **more reliable**, **more controllable**, and **better suited to production use cases**.

👉 [Learn more in the docs](https://docs.crewai.com/en/concepts/tasks#task-guardrails)